# PyTorch Fundamentals

In this notebook, I review basic tensor operations in PyTorch and building neural networks, mostly following Chapters 12 and 13 of <i>''Machine Learning with PyTorch and Scikit-Learn''</i> by Sebastian Raschka et al. This notebook covers:

1. Creating tensors in PyTorch
2. Manipulating data type and shape of tensor
3. Operations on tensors
4. Building input pipelines
5. Simple linear regression model in PyTorch
6. XOR classification with nn.Sequential
7. XOR classification with nn.Module

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

## 1. Creating tensors in PyTorch

Creating a tensor from a list or numpy array:

In [2]:
a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=np.int32)

t_a = torch.tensor(a)      # tensor from list
t_b = torch.from_numpy(b)  # tensor from numpy array

t_ones = torch.ones(2, 3)       # tensor of ones
rand_tensor = torch.rand(2, 3)  # tensor of random values

In [3]:
print(t_a)
print(t_b)
print(t_ones)
print(rand_tensor)

tensor([1, 2, 3])
tensor([4, 5, 6], dtype=torch.int32)
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.7794, 0.1164, 0.3139],
        [0.3971, 0.2793, 0.7157]])


## 2. Manipulating data type and shape of tensor

##### Change data type:

In [4]:
t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

torch.int64


##### Transposing a tensor:

In [5]:
t = torch.rand(3, 5)
print(t)
print(t.shape)

tensor([[0.1257, 0.9263, 0.1381, 0.8227, 0.7799],
        [0.8724, 0.3193, 0.8371, 0.5845, 0.2296],
        [0.1639, 0.9779, 0.6398, 0.3206, 0.7073]])
torch.Size([3, 5])


In [6]:
t_tr = torch.transpose(t, 0, 1)  # i.e. swapping dimensions 0 and 1
print(t_tr)
print(t_tr.shape)

tensor([[0.1257, 0.8724, 0.1639],
        [0.9263, 0.3193, 0.9779],
        [0.1381, 0.8371, 0.6398],
        [0.8227, 0.5845, 0.3206],
        [0.7799, 0.2296, 0.7073]])
torch.Size([5, 3])


##### Reshaping a tensor:

In [7]:
t = torch.zeros(30)
print(t)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])


In [8]:
t_reshape = t.reshape(5, 6)
print(t_reshape)
print(t_reshape.shape)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
torch.Size([5, 6])


##### Removing unnecessary dimensions:

In [9]:
t = torch.zeros(1, 2, 1, 5, 1)
print(t)
print(t.shape)

tensor([[[[[0.],
           [0.],
           [0.],
           [0.],
           [0.]]],


         [[[0.],
           [0.],
           [0.],
           [0.],
           [0.]]]]])
torch.Size([1, 2, 1, 5, 1])


In [10]:
t_sqz = torch.squeeze(t)
print(t_sqz)
print(t_sqz.shape)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
torch.Size([2, 5])


## 3. Operations on tensors

##### Instantiating random tensors

In [11]:
torch.manual_seed(1)
t1 = 2 * torch.rand(5, 2) - 1
t2 = torch.normal(mean=0, std=1, size=(5,2))  # Normally distributed random numbers

In [12]:
t1

tensor([[ 0.5153, -0.4414],
        [-0.1939,  0.4694],
        [-0.9414,  0.5997],
        [-0.2057,  0.5087],
        [ 0.1390, -0.1224]])

In [13]:
t2

tensor([[ 0.8590,  0.7056],
        [-0.3406, -1.2720],
        [-1.1948,  0.0250],
        [-0.7627,  1.3969],
        [-0.3245,  0.2879]])

##### Element-wise product

In [14]:
t3 = torch.multiply(t1, t2) 
t3

tensor([[ 0.4426, -0.3114],
        [ 0.0660, -0.5970],
        [ 1.1249,  0.0150],
        [ 0.1569,  0.7107],
        [-0.0451, -0.0352]])

##### Mean, sum, standard deviation

In [15]:
print(torch.mean(t1)) # Mean of all elements

tensor(0.0327)


In [16]:
print(torch.mean(t1, axis=0))  # Mean of each column (i.e. averaging over row dimension 0)

tensor([-0.1373,  0.2028])


In [17]:
print(torch.mean(t1, axis=1))   # Mean of each row (i.e. averaging over column dimension 1)

tensor([ 0.0369,  0.1378, -0.1709,  0.1515,  0.0083])


In [18]:
print(torch.sum(t1)) # Sum of all elements

tensor(0.3273)


In [19]:
print(torch.sum(t1, axis=0)) # Sum of each column (i.e. sum over each row dimension 0)

tensor([-0.6867,  1.0140])


In [20]:
print(torch.sum(t1, axis=1)) # Sum of each row (i.e. sum over each column dimension 1)

tensor([ 0.0739,  0.2755, -0.3417,  0.3030,  0.0166])


In [21]:
print(torch.std(t1, axis=0))

tensor([0.5378, 0.4591])


##### Matrix multiplication and norm

In [22]:
print(t1)
print(t1.shape)

tensor([[ 0.5153, -0.4414],
        [-0.1939,  0.4694],
        [-0.9414,  0.5997],
        [-0.2057,  0.5087],
        [ 0.1390, -0.1224]])
torch.Size([5, 2])


In [23]:
print(t2)
print(t2.shape)

tensor([[ 0.8590,  0.7056],
        [-0.3406, -1.2720],
        [-1.1948,  0.0250],
        [-0.7627,  1.3969],
        [-0.3245,  0.2879]])
torch.Size([5, 2])


Can calculate product of matrices: $t_1 \times t_2^T$, which will be a $5 \times 5$ matrix:

In [24]:
t3 = torch.matmul(t1, torch.transpose(t2, 0, 1))
print(t3)

tensor([[ 0.1312,  0.3860, -0.6267, -1.0096, -0.2943],
        [ 0.1647, -0.5310,  0.2434,  0.8035,  0.1980],
        [-0.3855, -0.4422,  1.1399,  1.5558,  0.4781],
        [ 0.1822, -0.5771,  0.2585,  0.8676,  0.2132],
        [ 0.0330,  0.1084, -0.1692, -0.2771, -0.0804]])


Can calculate product of matrices: $t_1^T \times t_2$, which will be a $2 \times 2$ matrix:

In [25]:
t4 = torch.matmul(torch.transpose(t1, 0, 1), t2)
print(t4)

tensor([[ 1.7453,  0.3392],
        [-1.6038, -0.2180]])


In [26]:
norm_t1 = torch.linalg.norm(t1, ord=2, dim=1)   # t1 is (5, 2) tensor. To calc norm of each row, sum of squares of elements in column direction (dim=1).
print(norm_t1)

tensor([0.6785, 0.5078, 1.1162, 0.5488, 0.1853])


In [27]:
np.sqrt(np.sum(np.square(t1.numpy()), axis=1)) # Check that normalized tensor above matches numpy result

array([0.67846215, 0.5078282 , 1.1162277 , 0.5487652 , 0.18525197],
      dtype=float32)

##### Split, stack, concatenate tensors

In [28]:
t = torch.rand(6)    
print(t)
t_splits = torch.chunk(t, 3)   # Split tensor t (with 6 elements) into m chunks with torch.chunk(t, m)
print(t_splits)

tensor([0.6397, 0.9743, 0.8300, 0.0444, 0.0246, 0.2588])
(tensor([0.6397, 0.9743]), tensor([0.8300, 0.0444]), tensor([0.0246, 0.2588]))


In [29]:
t = torch.rand(5)
print(t)
t_splits = torch.split(t, split_size_or_sections=[3, 2])   # Can instead specify size of each split with torch.split()
print(t_splits)

tensor([0.9391, 0.4167, 0.7140, 0.2676, 0.9906])
(tensor([0.9391, 0.4167, 0.7140]), tensor([0.2676, 0.9906]))


In [30]:
A = torch.ones(3)
print(A)

B = torch.zeros(2)
print(B)

tensor([1., 1., 1.])
tensor([0., 0.])


In [31]:
C = torch.cat([A, B], axis=0)  # Concatenate row-wise
print(C)

tensor([1., 1., 1., 0., 0.])


In [32]:
D = torch.ones(3)
print(D)

E = torch.zeros(3)
print(E)

tensor([1., 1., 1.])
tensor([0., 0., 0.])


In [33]:
F = torch.stack([D, E], axis=1)    # Concatenate tensors (D and E, both 1-dimensional) along a new dimension (column direction, dim=1)
print(F)                           # Now form a 2-dimensional tensor F

tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])
